In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from fastdtw import fastdtw

* Pseudocode of DTW

<div style="text-align: center;">
  <img src="./captures/dtw.png" alt="Pseudocode of DTW" style="display: inline-block;" />
</div>
* The great multivariate time series classification bake off: a review and experimental evaluation of recent algorithmic advances by Ruiz AFlynn MLarge J et al.

In [2]:
def dtw(s1, s2):
    """
    Compute the Dynamic Time Warping distance between two sequences.

    Args:
        s1 (numpy.ndarray): First sequence.
        s2 (numpy.ndarray): Second sequence.

    Returns:
        float: DTW distance between the two sequences.
    """
    n, m = len(s1), len(s2)
    dtw_matrix = np.zeros((n + 1, m + 1))

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = abs(s1[i - 1] - s2[j - 1])
            dtw_matrix[i, j] = cost + min(
                dtw_matrix[i - 1, j], dtw_matrix[i, j - 1], dtw_matrix[i - 1, j - 1]
            )

    return dtw_matrix[n, m]

In [11]:
# Load the Coffee dataset
coffee_train = pd.read_csv("data/Coffee/Coffee_TRAIN.txt", sep=r"\s+", header=None)
coffee_test = pd.read_csv("data/Coffee/Coffee_TEST.txt", sep=r"\s+", header=None)


X_train = coffee_train.iloc[:, 1:].values
y_train = coffee_train.iloc[:, 0].values
X_test = coffee_test.iloc[:, 1:].values
y_test = coffee_test.iloc[:, 0].values

n = len(X_train)
distance_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i + 1, n):
        distance_matrix[i, j] = dtw(X_train[i], X_train[j])
        distance_matrix[j, i] = distance_matrix[i, j]

# Train a k-NN classifier using DTW distances

knn = KNeighborsClassifier(n_neighbors=1, metric="precomputed")
knn.fit(distance_matrix, y_train)

# Evaluate the classifier on the test set
m = len(X_test)
X_test_distances = np.zeros((len(X_test), m))
for i in range(len(X_test)):
    for j in range(m):
        X_test_distances[i, j] = dtw(X_test[i], X_train[j])

accuracy = knn.score(X_test_distances, y_test)
print(f"Accuracy on the test set: {accuracy:.2f}")

Accuracy on the test set: 0.96


In [12]:
n = len(X_train)
distance_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i + 1, n):
        distance, _ = fastdtw(X_train[i], X_train[j])
        distance_matrix[i, j] = distance
        distance_matrix[j, i] = distance

# Train a k-NN classifier using DTW distances

knn = KNeighborsClassifier(n_neighbors=1, metric="precomputed")
knn.fit(distance_matrix, y_train)

# Evaluate the classifier on the test set
m = len(X_test)

X_test_distances = np.zeros((len(X_test), m))
for i in range(len(X_test)):
    for j in range(m):
        distance, _ = fastdtw(X_test[i], X_train[j])
        X_test_distances[i, j] = distance

accuracy = knn.score(X_test_distances, y_test)
print(f"Accuracy on the test set: {accuracy:.2f}")

Accuracy on the test set: 0.96
